#СПРИНТ 23 июля. ChatGPT + Dall-e

- Чаты и сообщения (in-memory)
- Отправка текста и изображений
- Генерация картинок через Dall-e
- Интеграция с OpenAI (chatGPT, Dall-e)
- Запуск через ngrok для доступа с фронта (Stackblitz)
---

##Библиотеки, импорты

###Установка библиотек

In [ ]:
# Установка зависимостей
!pip install fastapi uvicorn nest-asyncio pyngrok openai aiofiles --quiet

###Импорты и базовые настройки

In [ ]:
import openai
from openai import OpenAI
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional, Dict
from datetime import datetime
import shutil, os
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import aiofiles
import base64
from google.colab import userdata
import uuid
from typing import List, Optional

In [ ]:
# Для Colab: разрешаем повторный запуск uvicorn
nest_asyncio.apply()

# Папка для картинок
IMAGES_DIR = '/tmp/chat_images'
os.makedirs(IMAGES_DIR, exist_ok=True)

app = FastAPI()

# CORS для фронта
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],    allow_headers=['*'],
)

In [ ]:
# Получаем ключ из секретов Colab
try:
    api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = api_key

    # Теперь можно инициализировать клиент, он подхватит ключ из окружения
    client = openai.OpenAI()
    # или client = openai.OpenAI(api_key=api_key)

    print("✅ Ключ OpenAI успешно загружен из секретов.")

except Exception as e:
    print("🚨 Ошибка: Не удалось найти секрет 'OPENAI_API_KEY'.")
    print("👉 Убедитесь, что вы сохранили его в разделе 'Secrets' и включили 'Notebook access'.")

✅ Ключ OpenAI успешно загружен из секретов.


In [ ]:
# Получаем токен ngrok из секретов
!pkill ngrok
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTHTOKEN')

    # Используем f-string для безопасной передачи токена в команду
    !ngrok config add-authtoken {NGROK_TOKEN}

    print("✅ Токен ngrok успешно настроен из секретов.")

except Exception as e:
    print("🚨 Ошибка: Не удалось найти секрет 'NGROK_AUTHTOKEN'.")
    print("👉 Убедитесь, что вы сохранили его в разделе 'Secrets' и включили 'Notebook access'.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Токен ngrok успешно настроен из секретов.


### Pydantic-модели для запросов и ответов

In [ ]:
# In-memory хранилище
chats: Dict[str, dict] = {}  # chat_id -> chat dict
messages: Dict[str, List[dict]] = {}  # chat_id -> list of messages

# Pydantic модели
class Chat(BaseModel):
    id: str
    title: str

class Message(BaseModel):
    id: str
    chat_id: str
    role: str  # 'user' | 'assistant'
    type: str  # 'text' | 'image'
    content: str
    created_at: datetime

class DalleRequest(BaseModel):
    prompt: str

class DalleResponse(BaseModel):
    image_url: str

## Функции и эндпоинты

###Получение чатов

In [ ]:
def _get_all_chats_as_dtos() -> List[Chat]:
    """
    Извлекает все чаты из глобального хранилища и преобразует их
    в список объектов данных (DTO), подходящих для ответа клиенту.
    """
    print("\n-> Извлечение и форматирование списка чатов...")

    # List comprehension инкапсулирован здесь, в "грязной" зоне.
    chat_list = [Chat(id=cid, title=chat['title']) for cid, chat in chats.items()]

    print(f"   ...Найдено и подготовлено {len(chat_list)} чатов.")
    return chat_list

###Создание нового чата

In [ ]:
def _create_new_chat_session(title: str) -> Chat:
    """
    Создает новую сессию чата в глобальных хранилищах.

    - Генерирует уникальный ID.
    - Сохраняет название чата.
    - Инициализирует пустой список для сообщений.
    - Возвращает объект Chat, готовый для отправки клиенту.
    """
    print(f"\n-> Создание новой сессии чата с названием: '{title}'")

    chat_id = str(uuid.uuid4())

    # Манипуляции с глобальными переменными инкапсулированы здесь
    chats[chat_id] = {'title': title}
    messages[chat_id] = []

    new_chat_dto = Chat(id=chat_id, title=title)

    print(f"   ...Чат успешно создан. ID: {chat_id}")
    return new_chat_dto

###Текстовый запрос к GPT

In [ ]:
def _get_text_completion(chat_history: List[Dict]) -> Optional[str]:
    """Выполняет стандартный текстовый запрос к GPT-4o."""
    try:
        # Отправляем только текстовые сообщения для контекста
        text_messages = [
            {'role': m['role'], 'content': m['content']}
            for m in chat_history if m['type'] == 'text'
        ]
        if not text_messages:
            return None

        response = client.chat.completions.create(
            model='gpt-4o',
            messages=text_messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Ошибка при вызове OpenAI (text): {e}")
        return None

###Запрос к GPT с изображением

In [ ]:
def _get_vision_completion(image_path: str, user_prompt: str) -> Optional[str]:
    """Выполняет запрос к GPT-4o с изображением (Vision)."""
    try:
        with open(image_path, "rb") as f:
            image_bytes = f.read()
        image_b64 = base64.b64encode(image_bytes).decode()

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                    ]
                }
            ],
            max_tokens=300,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Ошибка при вызове OpenAI (vision): {e}")
        return None

###Сохранение загруженного файла

In [ ]:
async def _save_uploaded_image(image: UploadFile) -> str:
    """Сохраняет загруженный файл и возвращает путь к нему."""
    filename = f'{uuid.uuid4()}_{image.filename}'
    filepath = os.path.join(IMAGES_DIR, filename)
    async with aiofiles.open(filepath, 'wb') as out_file:
        content = await image.read()
        await out_file.write(content)
    return filepath

###Создание объекта сообщения

In [ ]:
def _create_message(chat_id: str, role: str, type: str, content: str) -> Dict:
    """Создает стандартизированный объект сообщения."""
    return {
        'id': str(uuid.uuid4()),
        'chat_id': chat_id,
        'role': role,
        'type': type,
        'content': content,
        'created_at': datetime.utcnow()
    }

###Обработка текстового сообщения

In [ ]:
async def _handle_text_message(chat_id: str, text: str, role: str, skip_gpt: bool):
    """Обрабатывает входящее текстовое сообщение."""
    if not text or not text.strip():
        print("Пустое текстовое сообщение, обработка пропущена.")
        return

    user_msg = _create_message(chat_id, role, 'text', text)
    messages[chat_id].append(user_msg)

    if not skip_gpt:
        assistant_text = _get_text_completion(messages[chat_id])
        if assistant_text:
            assistant_msg = _create_message(chat_id, 'assistant', 'text', assistant_text)
            messages[chat_id].append(assistant_msg)

###Обработка загруженного изображения

In [ ]:
async def _handle_uploaded_image(chat_id: str, image: UploadFile, text: Optional[str], role: str):
    """Обрабатывает загруженное изображение, сохраняет его, добавляет в историю и получает описание от Vision AI."""
    filepath = await _save_uploaded_image(image)
    image_url_for_client = f'/images/{os.path.basename(filepath)}'

    user_msg = _create_message(chat_id, role, 'image', image_url_for_client)
    messages[chat_id].append(user_msg)

    prompt = text or "Опиши, что изображено на картинке."
    assistant_text = _get_vision_completion(filepath, prompt)
    if assistant_text:
        assistant_msg = _create_message(chat_id, 'assistant', 'text', assistant_text)
        messages[chat_id].append(assistant_msg)
        print('Сообщение ассистента (vision) добавлено.')

###Обработка сообщения с URL изображения

In [ ]:
def _handle_image_url_message(chat_id: str, image_url: str, role: str):
    """
    Обрабатывает сообщение с URL изображения (например, от DALL-E).
    Просто добавляет его в историю чата без вызова AI.
    """
    user_msg = _create_message(chat_id, role, 'image', image_url)
    messages[chat_id].append(user_msg)
    print(f"Сообщение с URL изображения добавлено: {image_url}")

###Создание новой сессии для чата

In [ ]:
def _initialize_chat_session(chat_id: str) -> None:
    """
    Гарантирует, что для указанного ID чата существуют необходимые структуры данных.
    Если чат новый, создает для него пустой список сообщений.
    """
    if chat_id not in messages:
        print(f"   ...Создание новой сессии для чата '{chat_id}'.")
        messages[chat_id] = []

###Маршрутизатор обработчиков различных сообщений

In [ ]:
async def _dispatch_message_handler(
    chat_id: str,
    type: str,
    text: Optional[str],
    image: Optional[UploadFile],
    role: str,
    skip_gpt: bool
):
    """
    Выполняет роль 'маршрутизатора': определяет тип сообщения
    и вызывает соответствующий, более специфичный обработчик.
    """
    if type == 'text':
        print("   -> Маршрут: обработка текстового сообщения.")
        await _handle_text_message(chat_id, text, role, skip_gpt)

    elif type == 'image':
        if image:
            print("   -> Маршрут: обработка загруженного изображения.")
            await _handle_uploaded_image(chat_id, image, text, role)
        elif text:
            print("   -> Маршрут: обработка URL изображения.")
            _handle_image_url_message(chat_id, text, role)
        else:
            # Этот случай обрабатывается здесь, чтобы не усложнять эндпоинт.
            print("   (!) Предупреждение: Сообщение с типом 'image' не содержит ни файла, ни URL.")

    else:
        print(f"   (!) Ошибка: получен неизвестный тип сообщения '{type}'.")


###Генерация изображения

In [ ]:
def _generate_image_url_from_prompt(prompt: str) -> Optional[str]:
    """
    Выполняет вызов к API DALL-E для генерации изображения по промпту.

    В случае успеха возвращает URL сгенерированного изображения,
    в случае ошибки — логирует ее и возвращает None.
    """
    print(f"\n-> Отправка запроса в DALL-E с промптом: '{prompt[:70]}...'")
    try:
        response = client.images.generate(
            model="dall-e-2",
            prompt=prompt,
            n=1,
            size='512x512'
        )
        image_url = response.data[0].url
        print(f"   ...Изображение успешно сгенерировано. URL: {image_url}")
        return image_url
    except Exception as e:
        print(f"   (!) Ошибка при генерации в DALL-E: {e}")
        return None

###Поиск пути изображения

In [ ]:
def _find_image_filepath(filename: str) -> Optional[str]:
    """
    Ищет файл изображения в директории IMAGES_DIR.

    Возвращает полный путь к файлу, если он существует, иначе возвращает None.
    """
    print(f"\n-> Поиск изображения: '{filename}'")
    filepath = os.path.join(IMAGES_DIR, filename)

    if os.path.exists(filepath):
        print(f"   ...Файл найден: {filepath}")
        return filepath

    print(f"   (!) Файл не найден: {filename}")
    return None

###Эндпоинты

In [ ]:
# --- Эндпоинты ---

# Получение списка чатов с сервера
@app.get('/chats', response_model=List[Chat])
def get_chats():
    """
    Возвращает список всех существующих чатов.
    """
    # 1. Получить все чаты в готовом для отправки виде.
    all_chats = _get_all_chats_as_dtos()

    # 2. Вернуть их клиенту.
    return all_chats

# Создание чата
@app.post('/chats', response_model=Chat)
def create_chat(title: str = Form(...)):
    """
    Создает новый пустой чат с заданным названием.
    """
    # 1. Вызвать сервис для создания новой сессии чата.
    new_chat = _create_new_chat_session(title)

    # 2. Вернуть информацию о созданном чате клиенту.
    return new_chat

# Обработка сообщений (главный эндпоинт чата)
@app.post('/chats/{chat_id}/messages', response_model=List[Message])
async def post_message(
    chat_id: str,
    text: Optional[str] = Form(None),
    type: str = Form('text'),
    image: Optional[UploadFile] = File(None),
    skip_gpt: Optional[str] = Form(None),
    role: Optional[str] = Form(None)
):
    """
    Обрабатывает входящее сообщение, делегируя логику специализированным функциям.
    """
    print(f"\n-> Получен запрос в чат '{chat_id}' (тип: {type})")

    # 1. Подготовить сессию чата (создать, если нужно).
    _initialize_chat_session(chat_id)

    # 2. Выбрать правильный обработчик и выполнить всю логику.
    await _dispatch_message_handler(
        chat_id=chat_id,
        type=type,
        text=text,
        image=image,
        role=(role or 'user'),
        skip_gpt=bool(skip_gpt)
    )

    # 3. Вернуть клиенту актуальное состояние чата.
    print(f"   ...Отправка обновленной истории чата '{chat_id}'.")
    return messages.get(chat_id, [])

# Генерация изображений
@app.post('/dalle', response_model=DalleResponse)
def dalle_generate(req: DalleRequest):
    """
    Генерирует изображение по текстовому запросу, делегируя вызов API.
    """
    # 1. Получить URL изображения по текстовому промпту.
    generated_url = _generate_image_url_from_prompt(req.prompt)

    # 2. Сформировать и отправить ответ клиенту.
    # Если `generated_url` будет None (в случае ошибки), вернется пустая строка.
    return DalleResponse(image_url=(generated_url or ''))

# Доступ к изображению
@app.get('/images/{filename}')
def get_image(filename: str):
    """
    Возвращает статический файл изображения, если он существует.
    """
    # 1. Найти путь к файлу.
    filepath = _find_image_filepath(filename)

    # 2. Если файл найден, вернуть его. Иначе - выбросить ошибку 404.
    if filepath:
        return FileResponse(filepath)
    else:
        raise HTTPException(status_code=404, detail="Image not found")

## Запуск сервера

Полученную ссылку вставьте в переменную BACKEND_URL в конструкторе Stackblitz для запуска фронтенда

[Ссылка на интерфейс (файл App.js в песочнице stackblitz)](https://stackblitz.com/edit/vitejs-vite-cv3qpgno?file=src%2FApp.tsx)

In [ ]:
if __name__ == "__main__":
    nest_asyncio.apply()
    public_url = ngrok.connect(8000).public_url
    print("Ваш FastAPI backend доступен по адресу:", public_url)

    # Конфигурация и запуск Uvicorn
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

Ваш FastAPI backend доступен по адресу: https://708e98dffcaa.ngrok-free.app


INFO:     Started server process [144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



-> Создание новой сессии чата с названием: 'Чат 1'
   ...Чат успешно создан. ID: 8ddcd08d-225a-4eb1-97c3-3efb19413908
INFO:     45.12.254.157:0 - "POST /chats HTTP/1.1" 200 OK
INFO:     45.12.254.157:0 - "OPTIONS /chats/8ddcd08d-225a-4eb1-97c3-3efb19413908/messages HTTP/1.1" 200 OK

-> Получен запрос в чат '8ddcd08d-225a-4eb1-97c3-3efb19413908' (тип: text)
   -> Маршрут: обработка текстового сообщения.
Пустое текстовое сообщение, обработка пропущена.
   ...Отправка обновленной истории чата '8ddcd08d-225a-4eb1-97c3-3efb19413908'.
INFO:     45.12.254.157:0 - "POST /chats/8ddcd08d-225a-4eb1-97c3-3efb19413908/messages HTTP/1.1" 200 OK

-> Получен запрос в чат '8ddcd08d-225a-4eb1-97c3-3efb19413908' (тип: text)
   -> Маршрут: обработка текстового сообщения.
   ...Отправка обновленной истории чата '8ddcd08d-225a-4eb1-97c3-3efb19413908'.
INFO:     45.12.254.157:0 - "POST /chats/8ddcd08d-225a-4eb1-97c3-3efb19413908/messages HTTP/1.1" 200 OK

-> Получен запрос в чат '8ddcd08d-225a-4eb1-97c3-3ef

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [144]


KeyboardInterrupt: 